# From CMB simulations to power spectra analysis

In this tutorial, we will show how to generate CMB simulations from theoritical $C_\ell$, produce the corresponding CMB map tiles and finally analyze them using `psplay`. 

## CMB simulation

Using the attached $C_\ell$ [file](bode_almost_wmap5_lmax_1e4_lensedCls_startAt2.dat), we will first generate simulations with `pspy` in `CAR` pixellisation.

In [ ]:
from pspy import so_map

ncomp= 3
ra0, ra1, dec0, dec1 = -30, 30, -30, 30
res = 0.5

template = so_map.car_template(ncomp, ra0, ra1, dec0, dec1, res)

cl_file = "bode_almost_wmap5_lmax_1e4_lensedCls_startAt2.dat"
cmb = template.synfast(cl_file)

Then, we make 2 splits out of it, each with $5$ µK.arcmin rms in temperature and $5\times\sqrt{2}$ µK.arcmin in polarisation

In [ ]:
import numpy as np
nsplits = 2
splits = [cmb.copy() for i in range(nsplits)]
for i in range(nsplits):
    noise = so_map.white_noise(cmb, rms_uKarcmin_T=5, rms_uKarcmin_pol=np.sqrt(2)*5)
    splits[i].data += noise.data

We finally write on disk the two corresponding `fits` files

In [ ]:
for i in range(nsplits):
    splits[i].write_map("split{}_IQU_CAR.fits".format(i))

## Generation of tile files

We now have a total of 6 maps in `CAR` pixellisation we want to plot them into tile files. A tile file corresponds to a static `PNG` file representing a part of the sky at a given zoom level. To convert the maps, we will use a set of tools from `psplay`. 



### Conversion to tile files
You should get two files of ~ 1 Gb size. We should keep these files since we will use them later for the power spectra computation. Last step in the conversion process, we have to generate the different tiles 

In [ ]:
from psplay import tools

for i in range(nsplits):
    tools.car2tiles(input_file="split{}_IQU_car.fits".format(i), 
                    output_dir="tiles/split{}_IQU_car.fits".format(i))

At the end of the process, we get a new directory `tiles` with two sub-directories `split0_IQU_car.fits` and `split1_IQU_car_fits`. Within these two directories we have 6 directories which names refer to the zoom level : the directory `-5` correspond to the smallest zoom whereas the directory `0` corresponds to the most precise tiles. 

## Using `psplay` to visualize the maps

Now that we have generated the different tiles, we can interactively see the maps with `psplay`. The configuration of `psplay` can be either done using a dictionary or more easily with a `yaml`file. Basically, the configuration file needs to know where the original `FITS` files and the tile files are located. Other options can be set but we won't enter into too much details for this tutorial. 

For the purpose of this tutorial we have already created the `yaml` [file](simulation_to_analysis.yml) associated to the files we have created so far. Here is a copy-paste of it

```yaml
map:
  layers:
    tags:
      splits:
        values: [0, 1]
        keybindings: [j, k]

      components:
        values: [0, 1, 2]
        keybindings: [c, v]
        substitutes: ["T", "Q", "U"]


    tile_tmpl: files/tiles/split{splits}_IQU_car.fits/{z}/tile_{y}_{x}_{components}.png
    name_tmpl: CMB simulation - split {splits} - {components}

compute:
  maps:
    - id: split0
      file: split0_IQU_car.fits
    - id: split1
      file: split1_IQU_car.fits

plot:
  theory_file: bode_almost_wmap5_lmax_1e4_lensedCls_startAt2.dat
```

There are 3 mandatory section related to the 3 main steps namely the map visualization, the spectra computation and the graphical representation of the spectra. **The tricker part of this configuration is to set the path to tiles relatively to where your notebook/JupyterLab instance has been started**. We can't set an absolute path and so you have to make sure that your notebook has been initiated from the `examples` directory. Otherwise, you should change the path to tile files given that you have access to them from your JupyterLab instance. So make sure to initiate your JupyterLab session from a "top" directory.

We can now create an instance of `psplay` application and show the different maps

In [ ]:
from psplay import App
my_app = App("simulation_to_analysis.yml")
my_app.show_map()

If we unzoom enough, we will see the $\pm$ 30° patch size. We can also switch between the different I, Q, U and split layers. There are other options like the colormap and color scale which one can play and see how things change.

## Selecting sub-patches and computing the corresponding power spectra

Given the different map, we can now select patches by clicking on the square or the disk icons located just below the +/- zoom button. For instance, if we select a rectangle and another disk whose size are more or less the total size of our patch, we will get two surfaces of almost 3000 square degrees. Now we can ask `psplay` to compute the power spectra of both regions. Let's initiate the plot application

In [ ]:
my_app.show_plot()

We can now clik on the `Compute spectra` button and watch the log output by clicking on the `Logs` tab. Depending on your machine (mainly memory capacity), the process can be pretty long especially the transformation into spherical harmonics. It also depends of the size of your patches but within 1 or 2 minutes, we should get the final spectra for the different cross-correlation. For instance, we can switch between different combinations of spectra (TT, TE, EE,...). 

Finally, the `Configuration` tab offer different options like changing the $\ell_\mathrm{max}$ value of the method use for the computation.